# TOC

## Libraries

In [17]:
import re
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler


# Preprocessing function

In [3]:
def process_2019(df):
    # Basic Data Cleaning
    df.drop(columns=['TRAILING_LOAD_AFFECTED','TIMING_LOAD_AFFECTED', 'UNIT_CLASS_AFFECTED',
                     'INCIDENT_EQUIPMENT','TRUST_TRAIN_ID_REACT','TRUST_TRAIN_ID_RESP',
                     'FINANCIAL_YEAR_AND_PERIOD',
                     'PLANNED_DEST_WTT_DATETIME_AFF',
                     'PLANNED_ORIG_GBTT_DATETIME_AFF',
                     'PLANNED_ORIG_WTT_DATETIME_AFF',
                     'PLANNED_DEST_GBTT_DATETIME_AFF',
                     'PLANNED_DEST_WTT_DATETIME_AFF',
                     'TRAIN_SCHEDULE_TYPE_AFFECTED',
                     'TRACTION_TYPE_AFFECTED',
                     'INCIDENT_NUMBER',
                     'INCIDENT_CREATE_DATE', 'INCIDENT_START_DATETIME',
                     'INCIDENT_END_DATETIME', 'SECTION_CODE',
                     'NETWORK_RAIL_LOCATION_MANAGER', 'RESPONSIBLE_MANAGER',
                     'ATTRIBUTION_STATUS',
                     'INCIDENT_DESCRIPTION', 'REACTIONARY_REASON_CODE',
                     'INCIDENT_RESPONSIBLE_TRAIN', 'EVENT_DATETIME',
                     'TRAIN_SERVICE_CODE_AFFECTED','SERVICE_GROUP_CODE_AFFECTED',
                     'APP_TIMETABLE_FLAG_AFF', 'INCIDENT_REASON','START_STANOX','END_STANOX'], inplace=True)
    df.dropna(inplace=True)

    # Preprocessing of data
    data = df[['ENGLISH_DAY_TYPE', 'PERFORMANCE_EVENT_CODE', 'OPERATOR_AFFECTED']]
    ohe = OneHotEncoder(sparse=False).fit(data)

    df[ohe.get_feature_names_out()] = ohe.transform(data)
    df.drop(columns=['ENGLISH_DAY_TYPE', 'PERFORMANCE_EVENT_CODE', 'OPERATOR_AFFECTED','OPERATOR_AFFECTED_ZZ'], inplace=True)
    df['CANCELLATIONS'] = df['PERFORMANCE_EVENT_CODE_A'] = df['PERFORMANCE_EVENT_CODE_M']
    df['DELAYS'] = df['PERFORMANCE_EVENT_CODE_C'] + df['PERFORMANCE_EVENT_CODE_D'] + df['PERFORMANCE_EVENT_CODE_O'] + df['PERFORMANCE_EVENT_CODE_P'] + df['PERFORMANCE_EVENT_CODE_S'] + df['PERFORMANCE_EVENT_CODE_F']

    df.drop(columns=['PERFORMANCE_EVENT_CODE_A', 'PERFORMANCE_EVENT_CODE_C',
       'PERFORMANCE_EVENT_CODE_D', 'PERFORMANCE_EVENT_CODE_F',
       'PERFORMANCE_EVENT_CODE_M', 'PERFORMANCE_EVENT_CODE_O',
       'PERFORMANCE_EVENT_CODE_P', 'PERFORMANCE_EVENT_CODE_S'], inplace=True)

    return df


# Load first chunk of data 2018/2018

In [4]:
df = pd.read_csv("../raw_data/All Delays 2018-19.csv")


In [5]:
df.head()
df.dtypes


FINANCIAL_YEAR_AND_PERIOD          object
ORIGIN_DEPARTURE_DATE              object
TRUST_TRAIN_ID_AFFECTED            object
PLANNED_ORIG_LOC_CODE_AFF           int64
PLANNED_ORIG_GBTT_DATETIME_AFF     object
PLANNED_ORIG_WTT_DATETIME_AFF      object
PLANNED_DEST_LOC_CODE_AFFECTED      int64
PLANNED_DEST_GBTT_DATETIME_AFF     object
PLANNED_DEST_WTT_DATETIME_AFF      object
TRAIN_SERVICE_CODE_AFFECTED       float64
SERVICE_GROUP_CODE_AFFECTED        object
OPERATOR_AFFECTED                  object
ENGLISH_DAY_TYPE                   object
APP_TIMETABLE_FLAG_AFF             object
TRAIN_SCHEDULE_TYPE_AFFECTED       object
TRACTION_TYPE_AFFECTED             object
TRAILING_LOAD_AFFECTED            float64
TIMING_LOAD_AFFECTED               object
UNIT_CLASS_AFFECTED               float64
INCIDENT_NUMBER                     int64
INCIDENT_CREATE_DATE               object
INCIDENT_START_DATETIME            object
INCIDENT_END_DATETIME              object
SECTION_CODE                      

# Obtain pre-processed dataframe

In [6]:
clean_df = process_2019(df)


/Users/Debora/.pyenv/versions/3.10.6/envs/TrainDelays/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
clean_df.head()
clean_df.dtypes


ORIGIN_DEPARTURE_DATE              object
TRUST_TRAIN_ID_AFFECTED            object
PLANNED_ORIG_LOC_CODE_AFF           int64
PLANNED_DEST_LOC_CODE_AFFECTED      int64
PFPI_MINUTES                      float64
                                   ...   
OPERATOR_AFFECTED_XC              float64
OPERATOR_AFFECTED_XE              float64
OPERATOR_AFFECTED_XH              float64
CANCELLATIONS                     float64
DELAYS                            float64
Length: 68, dtype: object

#### 1) Try mapping the Stanox Codes of origin and destination ("PLANNED_ORIG_LOC_CODE_AFF) and ("PLANNED_DEST_LOC_CODE_AFFECTED) with this csv: https://github.com/alexfrancisross/NetworkRail/blob/master/Stanox-Station-Lookup.csv

In [8]:
mapping_df = pd.read_csv("../raw_data/Stanox-Station-Lookup.csv", sep = ";")
mapping_df.head()
null_coordinates_mask = (mapping_df['Latitude'].isnull()) & (mapping_df['Longitude'].isnull())
mapping_df = mapping_df[~null_coordinates_mask]
mapping_df = mapping_df[['Stanox','Latitude','Latitude']]


In [9]:
clean_df_lat_long_1 = pd.merge(mapping_df,
                      clean_df,
                      left_on ='Stanox',
                      right_on = 'PLANNED_ORIG_LOC_CODE_AFF' ,
                      how ='right')


In [10]:
print(f"Percentage of observations in the period 01 of 2018-2019 without longitude and latitude: {clean_df_lat_long_1.Stanox.isnull().sum()/clean_df_lat_long_1.shape[0]*100}")


Percentage of observations in the period 01 of 2018-2019 without longitude and latitude: 17.293403305383944


In [11]:
clean_df_lat_long_1.head()


,Stanox,Latitude,Latitude,ORIGIN_DEPARTURE_DATE,TRUST_TRAIN_ID_AFFECTED,PLANNED_ORIG_LOC_CODE_AFF,PLANNED_DEST_LOC_CODE_AFFECTED,PFPI_MINUTES,ENGLISH_DAY_TYPE_BH,ENGLISH_DAY_TYPE_SA,...,OPERATOR_AFFECTED_RZ,OPERATOR_AFFECTED_W1,OPERATOR_AFFECTED_WA,OPERATOR_AFFECTED_X1,OPERATOR_AFFECTED_X2,OPERATOR_AFFECTED_XC,OPERATOR_AFFECTED_XE,OPERATOR_AFFECTED_XH,CANCELLATIONS,DELAYS
0,1100.0,57.479852,57.479852,01-APR-18,011B44MN01,1100,4303,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1100.0,57.479852,57.479852,01-APR-18,011E17MF01,1100,54311,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1100.0,57.479852,57.479852,01-APR-18,011M16MZ01,1100,72410,50.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1100.0,57.479852,57.479852,01-APR-18,011M16MZ01,1100,72410,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1100.0,57.479852,57.479852,01-APR-18,011T80ML01,1100,6401,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


#### 2) Try mapping the Stanox Codes of origin and destination ("PLANNED_ORIG_LOC_CODE_AFF) and ("PLANNED_DEST_LOC_CODE_AFFECTED) with the scraped locations name; and then find the logitude/latitude on the basis of the location name.

In [30]:
stanox_locations_mapping = pd.read_csv("../raw_data/stanox_locations_mapping.csv")[['Location','STANOX']]
# ToDo keep only rows where the stanox code is made up of numbers
